## Pre-requisite
**Setup Ollama**<br>
For setting up ollama in your local environment, click [here](https://github.com/ollama/ollama).

**Pull Model**<br>
Pull the required model running the following command:
```bash
ollama pull llama3.2
```
To use more variants of the llama model check [here](https://ollama.com/library).

**Install python library**<br>
Install the python library for llama_index
```bash
pip install llama-index
pip install llama-index-embeddings-huggingface
pip install llama-index-llm-ollama
```
To learn more, click [here](https://docs.llamaindex.ai/en/stable/).

## Import Required Libraries

In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate

* SimpleDirectoryReader: Used for reading documents (specifically PDFs in this case) from a directory.
* HuggingFaceEmbedding: Interface to use pre-trained models from Hugging Face for generating embeddings (numerical representations of text).
* Settings: A configuration object where we can store models such as the LLM and embeddings for use later.
* VectorStoreIndex: Creates an index where documents are stored as vectors. This index will allow querying based on document similarity.
* Ollama: A class to load and use a large language model (LLM) for answering queries.
* PromptTemplate: Used to customize the prompt format for the LLM's response.

## Load the Knowledge Base (PDF Documents)

In [2]:
input_dir_path = "./pdf_directory"

loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".pdf"],
            recursive=True
        )
docs = loader.load_data()

Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)


* input_dir_path: Specifies the path where the PDF files are stored.
* SimpleDirectoryReader: Reads files from the directory specified. In this case, only files with the .pdf extension are processed (required_exts=[".pdf"]), and subdirectories are also searched (recursive=True).
* docs: After reading, the documents are loaded into memory and stored in the docs variable. These documents will later be embedded for similarity-based search.

Here, the knowledge base consists of research papers on LLM in Cybersecurity.

## Load Embedding Model

In [3]:
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

* HuggingFaceEmbedding: The chosen model for embedding is "BAAI/bge-large-en-v1.5", which is a large pre-trained embedding model that converts documents into embeddings (numerical vectors representing the semantic meaning of the text).
* trust_remote_code=True: This allows code from Hugging Face's model repository to be safely trusted and executed.

## Embed the Documents and Create a Vector Index

In [4]:
Settings.embed_model = embed_model
index = VectorStoreIndex.from_documents(docs)

C:\Users\jucse\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


* Settings.embed_model: The embedding model is assigned to Settings, making it accessible throughout the rest of the pipeline.
* VectorStoreIndex: This creates a vector-based index from the embedded documents. This index allows for efficient similarity-based searches using embeddings.
* from_documents(docs): Embeds the documents and stores them in a searchable vector format.

## Setup the LLM and Configure the Query Engine

In [5]:
llm = Ollama(model="llama3.2", request_timeout=120.0) 

Settings.llm = llm
query_engine = index.as_query_engine(streaming=True, similarity_top_k=4)

* Ollama: This loads the LLM, in this case, the "llama3.2" model, which is capable of generating text-based answers to queries.
* request_timeout=120.0: This sets a timeout of 120 seconds for generating a response from the LLM.
* Settings.llm: The LLM is stored in Settings for later use by the query engine.
* index.as_query_engine(): Converts the vector index into a query engine that allows you to retrieve relevant documents.
* streaming=True: The results are streamed progressively, so that there is no need to wait for the entire response.
* similarity_top_k=4: The query engine will return the top 4 most similar documents to the user’s query based on the vector embeddings.

## Create a Custom Prompt Template

In [6]:
qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information above I want you to think step by step to answer the query in a crisp manner, in case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)
query_engine.update_prompts({"response_synthesizer:text_qa_template": qa_prompt_tmpl})

* qa_prompt_tmpl_str: This is the string template for the LLM to follow when answering queries. It instructs the model to use the context (retrieved documents) to answer the query and to be clear in the response.
    * {context_str}: This placeholder will be filled with relevant document context retrieved from the vector index.
    * {query_str}: This placeholder will be filled with the user’s question.
* PromptTemplate: This wraps the string into a template that can be applied to each query.
* update_prompts(): The query engine's prompt for text-based question answering is updated with this custom prompt template.

## Query the Index

In [7]:
response1 = query_engine.query('How can we use LLM in cybersecurity?')
print(response1)

Based on the provided context information and the review paper's findings, here is a concise summary of how LLMs can be used in cybersecurity:

LLMs can be utilized in various cybersecurity tasks, including:

1. **Threat Intelligence**: Assisting in generating and analyzing Cyber Threat Intelligence (CTI), assisting in security response, and identifying potential threats.
2. **Vulnerability Detection**: Improving detection capabilities through prompt engineering and vulnerability dataset preparation.
3. **Secure Code Generation**: Evaluating the security of LLM-generated code, enhancing its security, and providing static analysis assistants.
4. **Program Repair**: Evaluating LLMs' program repair capability, combining LLMs with static analysis tools, and improving through different strategies.
5. **Anomaly Detection**: Assisting in log-based anomaly detection, web content security, digital forensic analysis, and other areas.
6. **LLM-Assisted Attack**: Enabling LLM-assisted privilege es

In [8]:
response2 = query_engine.query('What are the roles of Rule-based AI in designing critical infrastructure?')
print(response2)

Based on the provided context, here is a step-by-step analysis:

1. The context information discusses the importance of Critical Infrastructure (CI) and the need for effective cybersecurity solutions to protect it.
2. It mentions that Rule-based AI modeling has been identified as a promising approach in enhancing CI security solutions, particularly in terms of automation, intelligence, and transparency.
3. According to the article, Rule-based AI models can provide human-interpretable decisions, explainability, and trustworthiness in decision-making, which is essential in cybersecurity application areas.

Considering these points, I would answer that:

The roles of Rule-based AI in designing critical infrastructure include:
- Providing transparent and interpretable decisions for cybersecurity applications.
- Enabling automation and intelligence in threat detection, mitigation, prediction, diagnosis, and response.
- Enhancing explainability analysis to detect insider threats or suspiciou

* query(): This function sends a query to the vector index, retrieves the top-k similar documents (top 4 in this case), and then passes them to the LLM to generate a response.
* response1: This query asks, "How can we use LLM in cybersecurity?", and retrieves the relevant context from the PDFs before generating an answer.
* print(response1): The first response is printed out.
* response2: This query asks, "What are the roles of Rule-based AI in designing critical infrastructure?", and retrieves the relevant documents to formulate an answer.
* print(response2): The second response is printed out.

## Summary
The code sets up a Retrieval-Augmented Generation (RAG) pipeline using PDFs. It embeds the documents into vector space using Hugging Face models, creates a vector index, and queries the index using an LLM with a custom prompt template. When a user provides a question, the system retrieves relevant context from the documents and uses the LLM to generate a step-by-step answer based on that context.

To learn more visit [here](https://lightning.ai/lightning-ai/studios/rag-using-llama-3-2-by-meta-ai).